In [1]:
# import torch
# import pandas as pd
# import numpy as np
import time
import nltk
# from nltk.translate import bleu_score
from tqdm import tqdm
# from collections import Counter

In [2]:
from nltk.translate.bleu_score import corpus_bleu

In [3]:
from nltk import ngrams

In [5]:
# load the dataset
train_source_text_path = "./data/sup_train.en-fr.fr"
train_target_text_path = "./data/sup_train.en-fr.en"
dev_source_text_path = "./data/sup_valid.en-fr.fr"
dev_target_text_path = "./data/sup_valid.en-fr.en"

In [9]:
# read the data from the files and store them in pandas dataframe
def retrieve_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            data.append(line)
    return data

In [10]:
train_source_sentences = retrieve_data(train_source_text_path)
train_target_sentences = retrieve_data(train_target_text_path)

In [11]:
train_source_sentences = train_source_sentences[:len(train_source_sentences)//2]
train_target_sentences = train_target_sentences[:len(train_target_sentences)//2]

In [15]:
valid_source_sentences = retrieve_data(dev_source_text_path)
valid_target_sentences = retrieve_data(dev_target_text_path)
test_source_text_path = "./data/test.en-fr.fr"
test_source_sentences = retrieve_data(test_source_text_path)

In [12]:
# compute n-grams for each source sentence in the dictionary
n_grams_dict = {}
def create_src_ngrams(n):
    for sent in train_source_sentences:
        n_grams_dict[sent] = list(ngrams(sent.split(), n))

In [13]:
n = 3
create_src_ngrams(n)

In [16]:
def compute_ngram_overlap(input_sentence, n):
    max_overlap_count = float('-inf')
    best_translation_sent = ""
    
    input_sent_n_grams = list(ngrams(input_sentence.split(), n))
    for index, sent in enumerate(train_source_sentences):
        src_n_grams = n_grams_dict[sent]
        matches = len(set(src_n_grams).intersection(input_sent_n_grams))
        if matches > max_overlap_count:
            max_overlap_count = matches
            best_translation_sent = train_target_sentences[index]
    return max_overlap_count, best_translation_sent

In [26]:
compute_ngram_overlap(valid_source_sentences[0], n=3)

(5,
 'mr president , what the investigations into this incident show is what can best be described as a culture of management incompetence and one of complacency .')

In [18]:
all_predictions = []
all_targets = []
total_overlap = 0
total_examples = 0
for index, source_sent in tqdm(enumerate(valid_source_sentences), total=len(valid_source_sentences)):
    max_overlap_count, translation_sent = compute_ngram_overlap(source_sent, n=3)
    all_predictions.append(translation_sent)
    all_targets.append(valid_target_sentences[index])
    total_overlap += max_overlap_count
    total_examples += 1

average_overlap = total_overlap / total_examples
print(f"Average max n-gram overlap: {average_overlap}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [1:39:54<00:00,  3.00s/it]

Average max n-gram overlap: 5.313


In [19]:
references = [[target_sent.split()] for target_sent in all_targets]
candidates = [pred_sent.split() for pred_sent in all_predictions]
dev_bleu = corpus_bleu(references, candidates)
print('Dev BLEU score: ', dev_bleu)

Dev BLEU score:  0.06693695858184534
